In [1]:
import torch

In [2]:
print(f'Available GPU devices: {torch.cuda.device_count()}')

Available GPU devices: 1


In [3]:
##Importing dependencies

import gym
import gym_anytrading
from gym_anytrading.envs import TradingEnv, ForexEnv, StocksEnv, Actions, Positions

import pandas as pd
import numpy as np
np.set_printoptions(suppress=True, formatter={'float_kind':'{:f}'.format})

from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.cmd_util import make_vec_env

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

import os
import talib

import torch as th

from custom_rl_policy import CustomActorCriticPolicy
CUDA_LAUNCH_BLOCKING=1

c:\Users\hsain\.conda\envs\stock\lib\site-packages\stable_baselines3\common\cmd_util.py:6: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  "Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.", FutureWarning


In [4]:
df = pd.read_csv("SilverMicro.csv")

In [5]:
df.head(3)

,Date,Price,Open,High,Low,Volume,Chg%
0,"Mar 03, 2023","64,742","64,540","64,800","64,540",35.45K,0.65%
1,"Mar 02, 2023","64,324","64,627","64,781","64,161",118.71K,-0.69%
2,"Mar 01, 2023","64,774","64,950","65,280","64,639",154.67K,2.37%


In [6]:
df["Date"] = pd.to_datetime(df["Date"])

In [7]:
df.dtypes

Date      datetime64[ns]
Price             object
Open              object
High              object
Low               object
Volume            object
Chg%              object
dtype: object

In [8]:
df["Price"] = df["Price"].apply(lambda x: float(x.replace(",","")))
df["Open"] = df["Open"].apply(lambda x: float(x.replace(",","")))
df["High"] = df["High"].apply(lambda x: float(x.replace(",","")))
df["Low"] = df["Low"].apply(lambda x: float(x.replace(",","")))
df["Volume"] = df["Volume"].apply(lambda x: (x.replace(",","")))
df["Volume"] = df["Volume"].apply(lambda x: (x.replace("K","")))
df["Volume"] = df["Volume"].apply(lambda x: (float(x) * 1000))

In [9]:
df.dropna(inplace=True)

In [10]:
df.sort_values('Date', ascending=True, inplace=True)
df.head(4)

,Date,Price,Open,High,Low,Volume,Chg%
1824,2016-02-01,34880.0,34800.0,35062.0,34625.0,45000.0,0.33%
1823,2016-02-02,34804.0,34840.0,35091.0,34729.0,41870.0,-0.22%
1822,2016-02-03,35741.0,34865.0,35799.0,34850.0,73570.0,2.69%
1821,2016-02-04,35847.0,35622.0,35997.0,35561.0,48480.0,0.30%


In [11]:
new_df = df

In [12]:
new_df.head()

,Date,Price,Open,High,Low,Volume,Chg%
1824,2016-02-01,34880.0,34800.0,35062.0,34625.0,45000.0,0.33%
1823,2016-02-02,34804.0,34840.0,35091.0,34729.0,41870.0,-0.22%
1822,2016-02-03,35741.0,34865.0,35799.0,34850.0,73570.0,2.69%
1821,2016-02-04,35847.0,35622.0,35997.0,35561.0,48480.0,0.30%
1820,2016-02-05,35813.0,35870.0,36100.0,35540.0,54660.0,-0.09%


In [13]:
new_df.rename(columns={"Price":"Close"}, inplace=True)

In [14]:
# new_df.set_index("Date", inplace=True)

Custom indicator

In [15]:
# !pip install finta

In [16]:
from gym_anytrading.envs import StocksEnv
from finta import TA ##Technical indicator

In [17]:
# df["Volume"] = df["Volume"].apply(lambda x: float(x.replace(",","")))

In [18]:
new_df['sma'] = talib.SMA(new_df['Close'])

new_df['5sma'] = talib.SMA(new_df['Close'], timeperiod=5)

new_df['20sma'] = talib.SMA(new_df['Close'], timeperiod=20)

new_df['bb_upper'], new_df['bb_middle'], new_df['bb_lower'] = talib.BBANDS(new_df['Close'], matype=talib.MA_Type.T3)

new_df['bb_sell'] = (new_df['Close'] > new_df['bb_upper'])*1

new_df['bb_buy'] = (new_df['Close'] < new_df['bb_lower'])*1

new_df['bb_squeeze'] = (new_df['bb_upper'] - new_df['bb_lower'])/new_df['bb_middle']

new_df['mom'] = talib.MOM(new_df['Close'], timeperiod=10)

new_df['adx'] = talib.ADX(new_df['High'], new_df['Low'], new_df['Close'], timeperiod=10)

new_df['mfi'] = talib.MFI(new_df['High'], new_df['Low'], new_df['Close'], new_df['Volume'], timeperiod=10)

new_df['rsi'] = talib.RSI(new_df['Close'], timeperiod=10)


# new_df['trange'] = talib.TRANGE(new_df['High'], new_df['Low'], new_df['Close'])


# new_df['bop'] = talib.BOP(new_df['Open'], new_df['High'], new_df['Low'], new_df['Close'])

# new_df['cci'] = talib.CCI(new_df['High'], new_df['Low'], new_df['Close'], timeperiod=14)

new_df['STOCHRSI'] = talib.STOCHRSI(new_df['Close'],timeperiod=14,fastk_period=14,fastd_period=3,fastd_matype=0)[0]

slowk, slowd = talib.STOCH(new_df['High'], new_df['Low'], new_df['Close'], fastk_period=14,slowk_period=3,slowk_matype=0,slowd_period=3,slowd_matype=0)

macd, macdsignal, macdhist = talib.MACD(new_df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

new_df['slowk'] = slowk

new_df['slowd'] = slowd

new_df['macd'] = macd

new_df['macdsignal'] = macdsignal

new_df['macdhist'] = macdhist

new_df['NATR'] = talib.NATR(new_df['High'].ffill(), new_df['Low'].ffill(), new_df['Close'].ffill())

new_df['KAMA'] = talib.KAMA(new_df['Close'], timeperiod=10)

new_df['MAMA'], new_df['FAMA'] = talib.MAMA(new_df['Close'])

new_df['MAMA_buy'] = np.where((new_df['MAMA'] < new_df['FAMA']), 1, 0)

new_df['KAMA_buy'] = np.where((new_df['Close'] < new_df['KAMA']), 1, 0)

new_df['sma_buy'] = np.where((new_df['Close'] < new_df['5sma']), 1, 0)

new_df['maco'] = np.where((new_df['5sma'] < new_df['20sma']), 1, 0)

new_df['rsi_buy'] = np.where((new_df['rsi'] < 30), 1, 0)

new_df['rsi_sell'] = np.where((new_df['rsi'] > 70), 1, 0)

new_df['macd_buy_sell'] = np.where((new_df['macd'] < new_df['macdsignal']), 1, 0)

new_df.fillna(0, inplace=True)

In [19]:
new_df.set_index("Date", inplace=True)

In [20]:
new_df.columns

Index(['Close', 'Open', 'High', 'Low', 'Volume', 'Chg%', 'sma', '5sma',
       '20sma', 'bb_upper', 'bb_middle', 'bb_lower', 'bb_sell', 'bb_buy',
       'bb_squeeze', 'mom', 'adx', 'mfi', 'rsi', 'STOCHRSI', 'slowk', 'slowd',
       'macd', 'macdsignal', 'macdhist', 'NATR', 'KAMA', 'MAMA', 'FAMA',
       'MAMA_buy', 'KAMA_buy', 'sma_buy', 'maco', 'rsi_buy', 'rsi_sell',
       'macd_buy_sell'],
      dtype='object')

In [21]:
# new_df["SMA_5"] = TA.SMA(new_df, 5)
# new_df["SMA_20"] = TA.SMA(new_df, 20)
# new_df["SMA_50"] = TA.SMA(new_df, 50)
# new_df["SMA_200"] = TA.SMA(new_df, 200)
# bb_band= TA.BBANDS(new_df, 20)
# new_df = pd.concat([new_df, bb_band], axis=1)
# new_df["RSI"] = TA.RSI(new_df,10)
# macd = TA.VW_MACD(new_df)
# new_df = pd.concat([new_df, macd], axis=1)
# new_df['OBV'] = TA.OBV(new_df)
# new_df["S_RSI"] = TA.STOCHRSI(new_df)
# new_df.fillna(0, inplace=True)

In [22]:
## Create Env
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    # signal_features = env.df.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume','SMA_20','SMA_50','SMA_200','RSI', 'OBV','BB_UPPER','BB_MIDDLE','BB_LOWER','MACD','SIGNAL','S_RSI']].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume', 'sma', '5sma', '20sma',
       'bb_upper', 'bb_middle', 'bb_lower', 'bb_sell', 'bb_buy', 'bb_squeeze',
       'mom', 'adx', 'mfi', 'rsi', 'STOCHRSI', 'slowk', 'slowd', 'macd',
       'macdsignal', 'macdhist', 'NATR', 'KAMA', 'MAMA', 'FAMA', 'MAMA_buy',
       'KAMA_buy', 'sma_buy', 'maco', 'rsi_buy', 'rsi_sell', 'macd_buy_sell']].to_numpy()[start:end]
    return prices, signal_features


In [23]:
new_df.shape

(1825, 36)

In [24]:
from typing import Callable

def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [25]:
from stable_baselines3.common.callbacks import BaseCallback
class EntropyScheduleCallback(BaseCallback):
    """
    A callback class to adjust the entropy coefficient over time during training.
    """

    def __init__(self, ent_coef_schedule):
        super().__init__()
        self.ent_coef_schedule = ent_coef_schedule

    def _on_training_start(self) -> None:
        self.model.ent_coef = self.ent_coef_schedule(0)

    def _on_step(self) -> bool:
        t = self.num_timesteps
        self.model.ent_coef = self.ent_coef_schedule(t)
        return True

def ent_coef_schedule(t):
    """
    A custom function to adjust the entropy coefficient over time during training.
    """
    if t < 10000:
        return 0.1
    elif t < 20000:
        return 0.2
    else:
        return 0.3

In [26]:
class MyCustomEnv(StocksEnv):
    _process_data = add_signals #Native function _process_data
    
# env2 = MyCustomEnv(df=new_df, window_size=12, frame_bound=(43+20,1750))
env2 = MyCustomEnv(df=new_df, window_size=12, frame_bound=(1000,1750))

In [27]:
env2.signal_features[0]

array([45315.000000, 45687.000000, 44351.000000, 44459.000000,
       84560.000000, 45043.566667, 44640.800000, 44464.600000,
       45365.808079, 44591.269868, 43816.731658, 0.000000, 0.000000,
       0.034739, -355.000000, 18.852156, 45.635004, 46.041479, 40.704589,
       65.504258, 66.859278, -203.573513, -280.758432, 77.184919,
       1.567155, 44974.359468, 44572.978762, 44872.835939, 1.000000,
       1.000000, 1.000000, 0.000000, 0.000000, 0.000000, 0.000000])

In [28]:
env2.prices[0]

44351.0

In [29]:
# list of envs 
# num_envs = 3
# env_2 = [lambda: env2 for i in range(num_envs)]

# Vec Env 
# env_2 = SubprocVecEnv(env_2)
env_maker_2 = lambda: env2
# env_2 = DummyVecEnv([env_maker_2])
env_2 = make_vec_env(env_maker_2, n_envs=2, vec_env_cls=SubprocVecEnv)

In [30]:
log_path = os.path.join("Training","Logs")
# training_log_path = os.path.join(log_path, 'PPO_3')

In [31]:
entropy_schedule_callback = EntropyScheduleCallback(ent_coef_schedule)

In [34]:
# policy_kwargs = dict(activation_fn=th.nn.ReLU,
#                      net_arch=[dict(pi=[256, 128, 128, 64, 64], vf=[256, 128, 128, 64, 64])])

model_2 = PPO("MlpPolicy",env_2, verbose=1, tensorboard_log=log_path, device="cuda", learning_rate=linear_schedule(0.007), batch_size=256)
model_2.learn(total_timesteps=10000000, callback=entropy_schedule_callback)

Using cpu device


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 34681651200 bytes.

In [ ]:
env_3 = MyCustomEnv(df=new_df, window_size=12, frame_bound=(1750,1800))
# env_maker_2 = lambda: env_3
# env_3 = DummyVecEnv([env_maker_2])
# env_3 = make_vec_env(env_maker_2, n_envs=3)
# env_2 = make_vec_env(env_maker_2, n_envs=3, vec_env_cls=SubprocVecEnv)
obs = env_3.reset()
while True:
    obs = obs[np.newaxis, ...]
    action, _states = model_2.predict(obs)
    obs, reward, done, info = env_3.step(action)
    if done:
        print("info ", info)
        break
plt.figure(figsize=(50,16))
plt.cla()
env_3.render_all()
plt.show()

In [ ]:
env_maker = lambda: gym.make('stocks-v0', df=new_df, frame_bound=(30,100), window_size=30)
env = DummyVecEnv([env_maker])

In [ ]:
model.get_env().num_envs

In [ ]:
model = A2C("MlpLstmPolicy", env, verbose=1)

In [ ]:
model.learn(total_timesteps=100000)

In [ ]:
env = gym.make('stocks-v0', df=new_df, frame_bound=(2400,2443), window_size=30)
obs = env.reset()
while True:
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    if done:
        print("info ", info)
        break
plt.figure(figsize=(50,50))
plt.cla()
env.render_all()
plt.show()